In [1]:
import numpy as np
import pandas as pd
import os
import glob
import lightgbm as lgbm
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/optiver-realized-volatility-prediction/sample_submission.csv
/kaggle/input/optiver-realized-volatility-prediction/train.csv
/kaggle/input/optiver-realized-volatility-prediction/test.csv
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=97/888f813404d8417ca8d6b8aebd5f2951.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=43/bb0efa57f511470e817880842e3e2afa.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=21/1d8dc18ebfee47ffbb54b04e6afc0634.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=72/60f62a03d8854605901dda072c84db39.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=4/761268d671f9429abb29d9d2895e9bd2.parquet
/kaggle/input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=112/cd283097a5b54293ba400a19e811a7f9.parquet
/kaggle/input/optiver-realized-volatility-pr

In [2]:
train=pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train.head()
#裡面只有名稱 時間 target

,stock_id,time_id,target
0,0,5,0.004136
1,0,11,0.001445
2,0,16,0.002168
3,0,31,0.002195
4,0,62,0.001747


In [3]:
df0_book=pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id=0')
df0_book.head()

,time_id,seconds_in_bucket,bid_price1,ask_price1,bid_price2,ask_price2,bid_size1,ask_size1,bid_size2,ask_size2
0,5,0,1.001422,1.002301,1.00137,1.002353,3,226,2,100
1,5,1,1.001422,1.002301,1.00137,1.002353,3,100,2,100
2,5,5,1.001422,1.002301,1.00137,1.002405,3,100,2,100
3,5,6,1.001422,1.002301,1.00137,1.002405,3,126,2,100
4,5,7,1.001422,1.002301,1.00137,1.002405,3,126,2,100


# **Book_data**
* 時間 
* 最具/第二最具競爭力的購買水平的標準化價格
* 最具/第二最具競爭力的銷售水平的標準化價格
* 最具/第二最具競爭力的買入級別的股票數量。
* 最具/第二最具競爭力的賣出水平上的股票數量。



In [4]:
df0_trade=pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
df0_trade.head()

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1


# **Trade_data**
* 時間
* 一秒鐘內發生的已執行交易的平均價格。價格已標準化，平均值已按每筆交易中交易的股票數量加權。
* 交易的股票總數。
* 交易訂單的數量

--------------------------------------------------------------------------------

# What do we have?

* **買賣差價bid/ask spread**

由於不同股票在市場上的交易水平不同，我們採用最佳報價和最佳買入價的比率來計算買賣價差。
買賣價差的公式可以寫成以下形式：

𝐵𝑖𝑑𝐴𝑠𝑘𝑆𝑝𝑟𝑒𝑎𝑑=𝐵𝑒𝑠𝑡𝑂𝑓𝑓𝑒𝑟/𝐵𝑒𝑠𝑡𝐵𝑖𝑑−1 

* **加權平均價格(WAP)Weighted averaged price**

訂單簿也是股票估值的主要來源之一。
公平的帳面估值必須考慮兩個因素：訂單的水平和規模the level and the size of orders。
在本次比賽中，我們使用加權平均價格 (WAP) 來計算瞬時股票估值併計算已實現的波動率作為我們的**target**。
WAP的公式可以寫成如下，它考慮了最高價格和數量信息：

$WAP = \frac{BidPrice_{1}*AskSize_{1} + AskPrice_{1}*BidSize_{1}}{BidSize_{1} + AskSize_{1}}$
WAP=購買價格＊販賣規模＋販賣價格＊購買規模 / 購買規模＋販賣規模

* **小知識**

如果兩本書的買入價和賣出價分別處於相同的價格水平，則報價越多的書將產生較低的股票估值，因為訂單中有更多的潛在賣家，而更多的賣家意味著 市場上更多的供應**導致股票估值較低的事實**。


在大多數情況下，在連續交易時間內，訂單簿不應出現買單高於賣單或賣單的情況。 換句話說，最有可能的是，買入價和賣出價永遠不應該交叉。

* **對數收益率Log returns**

我們如何比較昨天和今天的股票價格？
我們可以通過將移動除以股票的起始價格來解決上述問題，有效地計算價格變化的百分比，也稱為股票收益stock return。


Log returns有幾個優點：
1. 他是可以隨時間增加的
2. 常規回報（regular returns）不能低於 -100%，而Log returns不受限制


Log returns在這邊我們寫成：
$r_{t_1, t_2} = \log \left( \frac{S_{t_2}}{S_{t_1}} \right)$

𝑆𝑡 = 𝑆在時間 𝑡 的股票價格
通常我們在看這10分鐘的對數收益率，我們寫成：$r_t = r_{t - 10 min, t}$

* **實際波動率Realized volatility**

當我們交易期權時，我們模型的一個有價值的輸入是**對數收益率的標準差standard deviation of the stock log returns**.。在更長或更短的時間間隔內計算的對數回報的標準差會有所不同，因此它通常被標準化為 1 年期，**年化標準差稱為波動率**。

在本次比賽中，您將獲得 10 分鐘的賬面數據，我們要求您預測接下來 10 分鐘的波動性。波動率將按如下方式測量：
$\sigma = \sqrt{\sum_{t}r_{t-1, t}^2}$

* **ＲＭＳＰＥ**

這個比賽的重點！

$\sqrt{\frac{1}{n}\sum^{n}_{i=1}((y_i-\hat{y_i})/y_i)^2}$

# **Feature Engineering**

In [5]:
def wap1(df):
    wap=(df['bid_price1']*df['ask_size1']+df['ask_price1']*df['bid_size1'])/(df['bid_size1']+df['ask_size1'])
    return wap
def wap2(df):
    wap=(df['bid_price2']*df['ask_size2']+df['ask_price2']*df['bid_size2'])/(df['bid_size2']+df['ask_size2'])
    return wap
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff()
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))
def count_unique(series):
    return len(np.unique(series))


In [6]:
#這裡我們來定義如何處理book裡的數據
def book_prep(path):
    df=pd.read_parquet(path)
    df['wap']=wap1(df)
    df['log_return']=df.groupby('time_id')['wap'].apply(log_return)
    df['wap2']=wap2(df)
    df['log_return2']=df.groupby('time_id')['wap2'].apply(log_return)
    df['price_spread']=(df['ask_price1']-df['bid_price1'])/(df['ask_price1']+df['bid_price1'])
    df['price_spread2']=(df['ask_price2']-df['bid_price2'])/(df['ask_price2']+df['bid_price2'])
    df['bid_spread']=df['bid_price1']-df['bid_price2']
    df['ask_spread']=df['ask_price1']-df['ask_price2']
    df['total_volume']=df['ask_size1'] + df['ask_size2'] + df['bid_size1'] + df['bid_size2']
    df['volume_imbalance']=abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    #建立一個用來聚合的字典
    create_feature_dict = {
        'log_return':[realized_volatility],
        'log_return2':[realized_volatility],
        'wap':[np.mean],
        'wap2':[np.mean],
        'price_spread':[np.mean],
        'price_spread2':[np.mean],
        'bid_spread':[np.mean],
        'ask_spread':[np.mean],
        'volume_imbalance':[np.mean],
        'total_volume':[np.mean],
            }
    #book裡的特徵建立完成，再來用groupby所有秒數
    df_feature=pd.DataFrame(df.groupby(['time_id']).agg(create_feature_dict)).reset_index()
    df_feature.columns=['_'.join(col) for col in df_feature.columns]#這裡會把time_id變成time_id_
    #
    last_seconds=[150,300,450]
    for second in last_seconds:
        second=600-second
        df_feature_sec=pd.DataFrame(df.query(f'seconds_in_bucket >={second}').groupby(['time_id']).agg(create_feature_dict)).reset_index()
        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns]
        df_feature_sec=df_feature_sec.add_suffix('_'+str(second))
        df_feature=pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature=df_feature.drop([f'time_id__{second}'],axis=1)
        
    #建立row_id
    stock_id=path.split('=')[1]
    df_feature['row_id']=df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature=df_feature.drop(['time_id_'],axis=1)
    return df_feature

In [7]:
%%time
data_dir = '../input/optiver-realized-volatility-prediction/'
path = data_dir + "book_train.parquet/stock_id=0"
book_prep(path)
#讓我們來查看執行book處理後的樣子，並看需要多少時間

CPU times: user 11.9 s, sys: 379 ms, total: 12.2 s
Wall time: 12.2 s


,log_return_realized_volatility,log_return2_realized_volatility,wap_mean,wap2_mean,price_spread_mean,price_spread2_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,...,log_return2_realized_volatility_150,wap_mean_150,wap2_mean_150,price_spread_mean_150,price_spread2_mean_150,bid_spread_mean_150,ask_spread_mean_150,volume_imbalance_mean_150,total_volume_mean_150,row_id
0,0.004499,0.006999,1.003725,1.003661,0.000426,0.000588,0.000176,-0.000151,134.894040,323.496689,...,0.006087,1.003832,1.003753,0.000429,0.000596,0.000188,-0.000147,123.586207,327.431034,0-5
1,0.001204,0.002476,1.000239,1.000206,0.000197,0.000335,0.000142,-0.000135,142.050000,411.450000,...,0.002262,1.000301,1.000245,0.000176,0.000310,0.000141,-0.000127,151.566474,419.277457,0-11
2,0.002369,0.004801,0.999542,0.999680,0.000363,0.000560,0.000197,-0.000198,141.414894,416.351064,...,0.004019,0.999126,0.999312,0.000340,0.000540,0.000161,-0.000241,132.084034,428.537815,0-16
3,0.002574,0.003637,0.998832,0.998633,0.000430,0.000579,0.000190,-0.000108,146.216667,435.266667,...,0.003273,0.998464,0.998353,0.000460,0.000600,0.000170,-0.000108,151.765432,424.234568,0-31
4,0.001894,0.003257,0.999619,0.999626,0.000199,0.000349,0.000191,-0.000109,123.846591,343.221591,...,0.002927,0.999618,0.999670,0.000198,0.000349,0.000187,-0.000117,131.474074,371.266667,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.002579,0.003821,0.997938,0.997864,0.000276,0.000409,0.000083,-0.000182,197.144781,374.235690,...,0.003507,0.997784,0.997666,0.000286,0.000405,0.000066,-0.000171,196.365639,349.916300,0-32751
3826,0.002206,0.002847,1.000310,1.000487,0.000271,0.000403,0.000092,-0.000172,233.781553,621.131068,...,0.002396,1.000528,1.000742,0.000289,0.000418,0.000079,-0.000181,253.850340,707.782313,0-32753
3827,0.002913,0.003266,0.999552,0.999456,0.000263,0.000405,0.000202,-0.000083,115.829787,343.734043,...,0.003006,0.999855,0.999793,0.000257,0.000407,0.000215,-0.000084,103.022556,313.142857,0-32758
3828,0.003046,0.005105,1.002357,1.002386,0.000240,0.000379,0.000113,-0.000166,132.074919,385.429967,...,0.004526,1.002281,1.002325,0.000259,0.000380,0.000095,-0.000148,120.972603,389.273973,0-32763


In [8]:
#先查看trade裡的資料
look=pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id=0')
look

,time_id,seconds_in_bucket,price,size,order_count
0,5,21,1.002301,326,12
1,5,46,1.002778,128,4
2,5,50,1.002818,55,1
3,5,57,1.003155,121,5
4,5,68,1.003646,4,1
...,...,...,...,...,...
123438,32767,471,0.998659,200,3
123439,32767,517,0.998515,90,1
123440,32767,523,0.998563,1,1
123441,32767,542,0.998803,90,4


In [9]:
#這裡我們利用trade裡的資料來建立特徵
def trade_prep(path):
    df = pd.read_parquet(path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    
    
    aggregate_dictionary = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean],
    }
    
    df_feature = df.groupby('time_id').agg(aggregate_dictionary).reset_index()
    
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]

    
    
    last_seconds = [150,300,450]
    
    for second in last_seconds:
        second = 600 - second
    
        df_feature_sec = df.query(f'seconds_in_bucket >= {second}').groupby('time_id').agg(aggregate_dictionary)
        df_feature_sec = df_feature_sec.reset_index()
        
        df_feature_sec.columns = ['_'.join(col) for col in df_feature_sec.columns]
        df_feature_sec = df_feature_sec.add_suffix('_' + str(second))
        
        df_feature = pd.merge(df_feature,df_feature_sec,how='left',left_on='time_id_',right_on=f'time_id__{second}')
        df_feature = df_feature.drop([f'time_id__{second}'],axis=1)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['trade_time_id_'],axis=1)
    
    return df_feature

In [10]:
%%time
data_dir = '../input/optiver-realized-volatility-prediction/'
path = data_dir + "trade_train.parquet/stock_id=0"
trade_prep(path)

CPU times: user 6.24 s, sys: 53.6 ms, total: 6.29 s
Wall time: 6.23 s


,trade_log_return_realized_volatility,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_log_return_realized_volatility_450,trade_seconds_in_bucket_count_unique_450,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,row_id
0,0.002006,40,3179,2.750000,0.001060,14.0,1042.0,2.642857,0.001308,21.0,1587.0,2.571429,0.001701,30.0,2069.0,2.433333,0-5
1,0.000901,30,1289,1.900000,0.000501,10.0,828.0,2.200000,0.000587,16.0,900.0,2.250000,0.000813,24.0,1173.0,2.041667,0-11
2,0.001961,25,2161,2.720000,0.001048,9.0,1085.0,3.666667,0.001137,12.0,1189.0,3.166667,0.001621,20.0,2010.0,2.950000,0-16
3,0.001561,15,1962,3.933333,0.000802,3.0,514.0,3.666667,0.001089,9.0,1556.0,5.111111,0.001401,11.0,1631.0,4.545455,0-31
4,0.000871,22,1791,4.045455,0.000360,4.0,43.0,3.500000,0.000453,11.0,1219.0,4.909091,0.000550,16.0,1570.0,4.500000,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.001519,52,3450,3.057692,0.000786,19.0,1159.0,2.947368,0.001162,35.0,2365.0,3.257143,0.001409,42.0,2957.0,3.238095,0-32751
3826,0.001411,28,4547,3.892857,0.000750,5.0,1158.0,4.600000,0.001066,12.0,2161.0,4.250000,0.001284,19.0,2494.0,3.421053,0-32753
3827,0.001521,36,4250,3.500000,0.000780,8.0,416.0,2.000000,0.001242,22.0,2294.0,3.727273,0.001375,27.0,2736.0,3.444444,0-32758
3828,0.001794,53,3217,2.150943,0.001012,12.0,1415.0,2.666667,0.001404,25.0,1627.0,1.920000,0.001650,36.0,2296.0,2.055556,0-32763


# Joblib
Joblib 是一組在 Python 中提供輕量級流水線的工具。特別是：

函數的透明磁盤緩存和延遲重新評估（記憶模式）
簡單的並行計算
Joblib 經過優化，特別是在大數據上快速且穩健，並且對numpy數組進行了特定優化。它是 BSD 許可的。
* 避免兩次計算相同的事情：代碼經常一次又一次地重新運行，例如在對計算量大的工作（如在科學開發中）進行原型設計時，但為緩解此問題而手工製作的解決方案容易出錯，並且經常導致不可重複的結果。
* 透明地持久化到磁盤：有效地持久化包含大數據的任意對像是很困難的。使用 joblib 的緩存機制避免了手寫持久化，並將磁盤上的文件隱式鏈接到原始 Python 對象的執行上下文。因此，joblib 的持久性有利於恢復應用程序狀態或計算作業，例如在崩潰後。

In [11]:
from joblib import Parallel, delayed
def preprocessor(list_stock_ids, is_train = True):
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        #先設定是否為train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
            
        df_tmp = pd.merge(book_prep(file_path_book),trade_prep(file_path_trade),on='row_id',how='left')
     
        return pd.concat([df,df_tmp])
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df

In [12]:
#測試一下，把stock0和stock1合併
list_stock_ids = [0,1]
preprocessor(list_stock_ids, is_train = True)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:   22.5s finished


,log_return_realized_volatility,log_return2_realized_volatility,wap_mean,wap2_mean,price_spread_mean,price_spread2_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,...,trade_size_sum_450,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150
0,0.004499,0.006999,1.003725,1.003661,0.000426,0.000588,0.000176,-0.000151,134.894040,323.496689,...,1042.0,2.642857,0.001308,21.0,1587.0,2.571429,0.001701,30.0,2069.0,2.433333
1,0.001204,0.002476,1.000239,1.000206,0.000197,0.000335,0.000142,-0.000135,142.050000,411.450000,...,828.0,2.200000,0.000587,16.0,900.0,2.250000,0.000813,24.0,1173.0,2.041667
2,0.002369,0.004801,0.999542,0.999680,0.000363,0.000560,0.000197,-0.000198,141.414894,416.351064,...,1085.0,3.666667,0.001137,12.0,1189.0,3.166667,0.001621,20.0,2010.0,2.950000
3,0.002574,0.003637,0.998832,0.998633,0.000430,0.000579,0.000190,-0.000108,146.216667,435.266667,...,514.0,3.666667,0.001089,9.0,1556.0,5.111111,0.001401,11.0,1631.0,4.545455
4,0.001894,0.003257,0.999619,0.999626,0.000199,0.000349,0.000191,-0.000109,123.846591,343.221591,...,43.0,3.500000,0.000453,11.0,1219.0,4.909091,0.000550,16.0,1570.0,4.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,0.003723,0.004996,1.000142,1.000006,0.000299,0.000436,0.000157,-0.000118,125.013029,296.185668,...,803.0,3.333333,0.001280,23.0,1889.0,3.608696,0.001585,36.0,2693.0,3.027778
7656,0.010829,0.012168,1.007503,1.007318,0.000461,0.000602,0.000159,-0.000125,254.006073,567.840081,...,13829.0,6.410256,0.006310,88.0,30858.0,8.136364,0.007915,145.0,62044.0,8.586207
7657,0.003135,0.004268,1.000854,1.000849,0.000324,0.000460,0.000141,-0.000132,163.645367,426.603834,...,487.0,2.400000,0.001567,11.0,980.0,2.727273,0.001804,21.0,1738.0,2.666667
7658,0.003750,0.005773,1.003032,1.003041,0.000211,0.000420,0.000190,-0.000231,138.235023,526.317972,...,4427.0,2.656250,0.001919,57.0,8274.0,2.701754,0.002557,87.0,12238.0,2.643678


*  Training_data

In [13]:
train = pd.read_csv(data_dir + 'train.csv')
train_ids = train.stock_id.unique()

In [14]:
%%time
df_train = preprocessor(list_stock_ids= train_ids, is_train = True)
#這裡我們把整個訓練資料合併起來

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  6.8min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed: 17.3min finished


CPU times: user 4.58 s, sys: 397 ms, total: 4.98 s
Wall time: 17min 19s


In [15]:
#把feature和target合併
train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
train = train[['row_id','target']]
df_train = train.merge(df_train, on = ['row_id'], how = 'left')

*  Test_data

In [16]:
test = pd.read_csv(data_dir + 'test.csv')
test_ids = test.stock_id.unique()

In [17]:
%%time
df_test = preprocessor(list_stock_ids= test_ids, is_train = False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


CPU times: user 22.9 ms, sys: 1.9 ms, total: 24.8 ms
Wall time: 280 ms


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.3s finished


In [18]:
df_test = test.merge(df_test, on = ['row_id'], how = 'left')

# Ready To Train

In [19]:
from sklearn.model_selection import KFold
df_train['stock_id']=df_train['row_id'].apply(lambda x:x.split('-')[0])
df_test['stock_id']=df_test['row_id'].apply(lambda x:x.split('-')[0])

stock_id_target_mean=df_train.groupby('stock_id')['target'].mean()
df_test['stock_id_target_enc']=df_test['stock_id'].map(stock_id_target_mean)

tmp=np.repeat(np.nan,df_train.shape[0])
kf=KFold(n_splits=10,shuffle=True)
for idx_1,idx_2 in kf.split(df_train):
    target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()
    tmp[idx_2]=df_train['stock_id'].iloc[idx_2].map(target_mean)
df_train['stock_id_target_enc'] = tmp

In [20]:
#這裡我們把stock_id改成int
df_train['stock_id'] = df_train['stock_id'].astype(int)
df_test['stock_id'] = df_test['stock_id'].astype(int)

In [21]:
X=df_train.drop(['row_id','target'],axis=1)
y=df_train['target']

In [22]:
#定義RMSPE
def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def feval_RMSPE(preds, lgbm_train):
    labels = lgbm_train.get_label()
    return 'RMSPE', round(rmspe(y_true = labels, y_pred = preds),5), False
#定義模型用的超參數
params = {
      "objective": "rmse", 
      "metric": "rmse", 
      "boosting_type": "gbdt",
      'early_stopping_rounds': 30,
      'learning_rate': 0.01,
      'lambda_l1': 1,
      'lambda_l2': 1,
  }

In [23]:
#這裡我們設定Cross Vaildation需要用的前置作業
kf = KFold(n_splits=4,shuffle=True)
oof = pd.DataFrame()        
models = []                  
scores = 0.0          

In [24]:
#開始訓練
for fold, (trn_idx, val_idx) in enumerate(kf.split(X, y)):

    print("Fold :", fold+1)
    
    # 建立訓練和驗證資料
    X_train, y_train = X.loc[trn_idx], y[trn_idx]
    X_valid, y_valid = X.loc[val_idx], y[val_idx]
    
    #RMSPE 權重
    #https://www.kaggle.com/c/optiver-realized-volatility-prediction/discussion/250324
    #根據大神，我們利用RMSPE weight
    weights = 1/np.square(y_train)
    lgbm_train = lgbm.Dataset(X_train,y_train,weight = weights)

    weights = 1/np.square(y_valid)
    lgbm_valid = lgbm.Dataset(X_valid,y_valid,reference = lgbm_train,weight = weights)
    
    # 構建模型
    model = lgbm.train(params=params,
                      train_set=lgbm_train,
                      valid_sets=[lgbm_train, lgbm_valid],
                      num_boost_round=1000,         
                      feval=feval_RMSPE,
                      verbose_eval=200,
                      categorical_feature = ['stock_id']                
                     )
    
    # 預測valid裡的資料
    y_pred = model.predict(X_valid, num_iteration=model.best_iteration)
    #這裡我們顯示出每次訓練後RMSPE的分數
    RMSPE = round(rmspe(y_true = y_valid, y_pred = y_pred),4)
    print(f'Performance of the　prediction: , RMSPE: {RMSPE}')

    #記錄分數與模型
    scores += RMSPE / 4
    models.append(model)
    print("*" * 100)
    

Fold : 1


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.109444 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14543
[LightGBM] [Info] Number of data points in the train set: 321699, number of used features: 58


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 0.001798
Training until validation scores don't improve for 30 rounds
[200]	training's rmse: 0.000532913	training's RMSPE: 0.2468	valid_1's rmse: 0.000542461	valid_1's RMSPE: 0.25057
[400]	training's rmse: 0.000498957	training's RMSPE: 0.23107	valid_1's rmse: 0.000511961	valid_1's RMSPE: 0.23648
[600]	training's rmse: 0.000491918	training's RMSPE: 0.22781	valid_1's rmse: 0.000508791	valid_1's RMSPE: 0.23502
[800]	training's rmse: 0.000487177	training's RMSPE: 0.22562	valid_1's rmse: 0.000507091	valid_1's RMSPE: 0.23424
[1000]	training's rmse: 0.000483288	training's RMSPE: 0.22382	valid_1's rmse: 0.000506339	valid_1's RMSPE: 0.23389
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.000483288	training's RMSPE: 0.22382	valid_1's rmse: 0.000506339	valid_1's RMSPE: 0.23389
Performance of the　prediction: , RMSPE: 0.2339
****************************************************************************************************
Fold 

...

[LightGBM] [Info] Start training from score 0.001800
Training until validation scores don't improve for 30 rounds
[200]	training's rmse: 0.000534643	training's RMSPE: 0.24742	valid_1's rmse: 0.000538155	valid_1's RMSPE: 0.24914
[400]	training's rmse: 0.000500675	training's RMSPE: 0.2317	valid_1's rmse: 0.000507346	valid_1's RMSPE: 0.23488
[600]	training's rmse: 0.000493501	training's RMSPE: 0.22838	valid_1's rmse: 0.000503548	valid_1's RMSPE: 0.23312
Early stopping, best iteration is:
[626]	training's rmse: 0.000492818	training's RMSPE: 0.22806	valid_1's rmse: 0.00050323	valid_1's RMSPE: 0.23297
Performance of the　prediction: , RMSPE: 0.233
****************************************************************************************************
Fold : 4


/opt/conda/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1706: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['stock_id']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.155127 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14543
[LightGBM] [Info] Number of data points in the train set: 321699, number of used features: 58


/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1433: UserWarning: Overriding the parameters from Reference Dataset.
  _log_warning('Overriding the parameters from Reference Dataset.')
/opt/conda/lib/python3.7/site-packages/lightgbm/basic.py:1245: UserWarning: categorical_column in param dict is overridden.
  _log_warning('{} in param dict is overridden.'.format(cat_alias))


[LightGBM] [Info] Start training from score 0.001806
Training until validation scores don't improve for 30 rounds
[200]	training's rmse: 0.000533539	training's RMSPE: 0.2465	valid_1's rmse: 0.000554077	valid_1's RMSPE: 0.25778
[400]	training's rmse: 0.000499746	training's RMSPE: 0.23088	valid_1's rmse: 0.000523586	valid_1's RMSPE: 0.24359
[600]	training's rmse: 0.000492781	training's RMSPE: 0.22767	valid_1's rmse: 0.000519206	valid_1's RMSPE: 0.24155
[800]	training's rmse: 0.000488027	training's RMSPE: 0.22547	valid_1's rmse: 0.000517226	valid_1's RMSPE: 0.24063
[1000]	training's rmse: 0.00048427	training's RMSPE: 0.22373	valid_1's rmse: 0.000515892	valid_1's RMSPE: 0.24001
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 0.00048427	training's RMSPE: 0.22373	valid_1's rmse: 0.000515892	valid_1's RMSPE: 0.24001
Performance of the　prediction: , RMSPE: 0.24
****************************************************************************************************


In [25]:
#看個成果吧
scores

0.2351

In [26]:
#準備測試資料
y_pred = df_test[['row_id']]
X_test = df_test.drop(['time_id', 'row_id'], axis = 1)
X_test

,stock_id,log_return_realized_volatility,log_return2_realized_volatility,wap_mean,wap2_mean,price_spread_mean,price_spread2_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,...,trade_order_count_mean_450,trade_log_return_realized_volatility_300,trade_seconds_in_bucket_count_unique_300,trade_size_sum_300,trade_order_count_mean_300,trade_log_return_realized_volatility_150,trade_seconds_in_bucket_count_unique_150,trade_size_sum_150,trade_order_count_mean_150,stock_id_target_enc
0,0,0.000294,0.000252,1.000405,1.00055,0.000279,0.000533,0.000393,-0.000115,164.666667,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028
2,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.004028


In [27]:
target = np.zeros(len(X_test))
#使用各個模型來預測test，並使用最好的模型
for model in models:
    pred = model.predict(X_test[X_valid.columns], num_iteration=model.best_iteration)
    target += pred / len(models)

In [28]:
y_pred = y_pred.assign(target = target)
y_pred

,row_id,target
0,0-4,0.001005
1,0-32,0.001053
2,0-34,0.001053


In [29]:
y_pred.to_csv('submission.csv',index = False)